In [1]:
import numpy as np 
import pandas as pd
import os

# Pre-Processing
Here, we pre-process data in the email bodies.

### Read in the manipulated, unlabeled data

In [2]:
train = pd.read_csv("./enron/unlabeled/train.csv")
train.head()

,Message-ID,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user
0,<30054600.1075841565738.JavaMail.evans@thyme>,2002-01-31 03:37:11,frozenset({'pete.davis@enron.com'}),frozenset({'pete.davis@enron.com'}),Start Date: 1/30/02; HourAhead hour: 22;,Schedule Crawler<pete.davis@enron.com>@ENRON,"Davis, Pete </O=ENRON/OU=NA/CN=RECIPIENTS/CN=P...","Meyers, Albert </O=ENRON/OU=NA/CN=RECIPIENTS/C...",NaN,"\ExMerge - Solberg, Geir\Deleted Items",SOLBERG-G,geir solberg 6-26-02.PST,\r\r\n\r\r\nStart Date: 1/30/02; HourAhead hou...,solberg-g
1,<33136589.1075852829115.JavaMail.evans@thyme>,2001-10-12 03:36:25,frozenset({'402075.57130981.1@1.americanexpres...,frozenset({'kpresto@enron.com'}),Help the Sept. 11 disaster relief effort,Membership Rewards <membershiprewards+402075.5...,kpresto@enron.com,NaN,NaN,\KPRESTO (Non-Privileged)\Deleted Items,Presto-K,KPRESTO (Non-Privileged).pst,----------------------------------------------...,presto-k
2,<4916370.1075861400181.JavaMail.evans@thyme>,2001-11-05 14:37:15,frozenset({'harry.arora@enron.com'}),frozenset({'iris.mack@enron.com'}),FW: Synthetic Peaker,"Arora, Harry </O=ENRON/OU=NA/CN=RECIPIENTS/CN=...","Mack, Iris </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Im...",NaN,NaN,"\HARORA (Non-Privileged)\Arora, Harry\Sent Items",Arora-H,HARORA (Non-Privileged).pst,\r\r\n-----Original Message-----\r\r\nFrom: B...,arora-h
3,<24276539.1075856274091.JavaMail.evans@thyme>,2000-10-19 20:25:00,frozenset({'enron.announcements@enron.com'}),frozenset({'all_ena_egm_eim@enron.com'}),Ameriflash Newsletter,Enron Announcements,All_ENA_EGM_EIM,NaN,NaN,\Vincent_Kaminski_Jun2001_1\Notes Folders\All ...,Kaminski-V,vkamins.nsf,NOTE FROM MARK FREVERT\r\r\n\r\r\nWith the wid...,kaminski-v
4,<28413439.1075854360990.JavaMail.evans@thyme>,2001-04-05 13:17:00,frozenset({'darron.giron@enron.com'}),frozenset({'phillip.love@enron.com'}),FW: the mullet!:,Darron C Giron,Phillip M Love,NaN,NaN,\Darron_Giron_Jun2001\Notes Folders\All documents,Giron-D,dgiron.nsf,---------------------- Forwarded by Darron C G...,giron-d
